# Predicción de Tendencias de Hashtags

## Problema Real
¿Cuál hashtag va a explotar la próxima semana? ¿Vale la pena invertir contenido ahí?

## Objetivo
Predecir el volumen de posts de diferentes hashtags y identificar tendencias emergentes para que las marcas puedan aprovecharlas antes que la competencia.

## Metodología
1. Análisis de datos históricos de hashtags
2. Regresión lineal para predicción de volumen
3. Identificación de tendencias (subida vs bajada)
4. Relación con eventos del mundo real

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 2. Generación de Datos Sintéticos

Generaremos datos simulados de hashtags populares con:
- Volumen de posts por día
- Engagement promedio (likes, comments, shares)
- Eventos relacionados

In [ ]:
# Definir fechas (últimos 60 días)
np.random.seed(42)
fecha_fin = datetime.now()
fecha_inicio = fecha_fin - timedelta(days=60)
fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

# Hashtags a analizar
hashtags = ['#TechTrends', '#SustainableLiving', '#AIRevolution', 
            '#HealthyLifestyle', '#DigitalMarketing', '#Startup',
            '#RemoteWork', '#CryptoNews', '#FitnessGoals', '#TravelDiaries']

# Eventos importantes (influyen en algunos hashtags)
eventos = {
    fecha_inicio + timedelta(days=10): {'nombre': 'Conferencia Tech Global', 'hashtags': ['#TechTrends', '#AIRevolution']},
    fecha_inicio + timedelta(days=25): {'nombre': 'Día Mundial de la Salud', 'hashtags': ['#HealthyLifestyle', '#FitnessGoals']},
    fecha_inicio + timedelta(days=40): {'nombre': 'Lanzamiento Bitcoin ETF', 'hashtags': ['#CryptoNews']},
    fecha_inicio + timedelta(days=50): {'nombre': 'Black Friday', 'hashtags': ['#DigitalMarketing', '#Startup']}
}

print(f"Período de análisis: {fecha_inicio.date()} a {fecha_fin.date()}")
print(f"Total de días: {len(fechas)}")
print(f"Hashtags analizados: {len(hashtags)}")

In [ ]:
# Generar datos para cada hashtag
datos = []

for hashtag in hashtags:
    # Tendencia base (algunas crecen, otras decrecen)
    if hashtag in ['#AIRevolution', '#CryptoNews', '#RemoteWork']:
        tendencia = np.linspace(1000, 5000, len(fechas))  # Tendencia creciente
    elif hashtag in ['#TravelDiaries']:
        tendencia = np.linspace(3000, 1500, len(fechas))  # Tendencia decreciente
    else:
        tendencia = np.linspace(2000, 2500, len(fechas))  # Tendencia estable
    
    for i, fecha in enumerate(fechas):
        # Volumen base con ruido
        volumen_base = tendencia[i] + np.random.normal(0, 300)
        
        # Incremento por eventos
        incremento_evento = 0
        if fecha in eventos and hashtag in eventos[fecha]['hashtags']:
            # Pico en el evento y días siguientes
            incremento_evento = 3000
        
        # Verificar días cercanos al evento (efecto de 5 días)
        for fecha_evento, info_evento in eventos.items():
            if hashtag in info_evento['hashtags']:
                dias_diff = abs((fecha - fecha_evento).days)
                if dias_diff <= 5:
                    incremento_evento += max(0, (3000 - dias_diff * 500))
        
        volumen = max(0, volumen_base + incremento_evento)
        
        # Engagement correlacionado con volumen
        engagement_promedio = (volumen * 0.05) + np.random.normal(0, 50)
        engagement_promedio = max(0, engagement_promedio)
        
        # Identificar si hay evento
        evento_nombre = None
        if fecha in eventos and hashtag in eventos[fecha]['hashtags']:
            evento_nombre = eventos[fecha]['nombre']
        
        datos.append({
            'fecha': fecha,
            'hashtag': hashtag,
            'volumen_posts': int(volumen),
            'engagement_promedio': int(engagement_promedio),
            'evento': evento_nombre
        })

# Crear DataFrame
df = pd.DataFrame(datos)
print("\nDataset creado exitosamente!")
print(f"Total de registros: {len(df)}")
df.head(10)

## 3. Exploración de Datos (EDA)

In [ ]:
# Estadísticas descriptivas
print("=" * 60)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 60)
print(df.describe())

print("\n" + "=" * 60)
print("INFORMACIÓN DEL DATASET")
print("=" * 60)
print(df.info())

In [ ]:
# Visualizar volumen de posts por hashtag a lo largo del tiempo
plt.figure(figsize=(15, 8))

for hashtag in hashtags:
    df_hashtag = df[df['hashtag'] == hashtag]
    plt.plot(df_hashtag['fecha'], df_hashtag['volumen_posts'], 
             marker='o', markersize=3, label=hashtag, linewidth=2)

# Marcar eventos
for fecha_evento, info in eventos.items():
    plt.axvline(x=fecha_evento, color='red', linestyle='--', alpha=0.3)
    plt.text(fecha_evento, plt.ylim()[1] * 0.95, info['nombre'], 
             rotation=90, verticalalignment='top', fontsize=8)

plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Volumen de Posts', fontsize=12)
plt.title('Evolución del Volumen de Posts por Hashtag', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Promedio de volumen y engagement por hashtag
resumen_hashtags = df.groupby('hashtag').agg({
    'volumen_posts': ['mean', 'std', 'min', 'max'],
    'engagement_promedio': 'mean'
}).round(2)

resumen_hashtags.columns = ['Volumen_Promedio', 'Volumen_StdDev', 'Volumen_Min', 'Volumen_Max', 'Engagement_Promedio']
resumen_hashtags = resumen_hashtags.sort_values('Volumen_Promedio', ascending=False)

print("\n" + "=" * 60)
print("RESUMEN POR HASHTAG")
print("=" * 60)
print(resumen_hashtags)

In [ ]:
# Visualización de comparación de hashtags
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Volumen promedio
resumen_hashtags['Volumen_Promedio'].plot(kind='barh', ax=axes[0], color='skyblue')
axes[0].set_xlabel('Volumen Promedio de Posts', fontsize=11)
axes[0].set_title('Volumen Promedio por Hashtag', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Engagement promedio
resumen_hashtags['Engagement_Promedio'].plot(kind='barh', ax=axes[1], color='lightcoral')
axes[1].set_xlabel('Engagement Promedio', fontsize=11)
axes[1].set_title('Engagement Promedio por Hashtag', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Preparación de Datos para Modelo

In [ ]:
# Crear características para el modelo
df['dia_numero'] = (df['fecha'] - df['fecha'].min()).dt.days
df['dia_semana'] = df['fecha'].dt.dayofweek
df['es_evento'] = df['evento'].notna().astype(int)

# Codificar hashtags
df['hashtag_encoded'] = pd.Categorical(df['hashtag']).codes

print("Características creadas:")
print(df[['fecha', 'hashtag', 'dia_numero', 'dia_semana', 'es_evento', 'hashtag_encoded', 'volumen_posts']].head())

## 5. Modelo de Regresión Lineal para Predicción

In [ ]:
# Preparar variables X e y
X = df[['dia_numero', 'dia_semana', 'es_evento', 'hashtag_encoded', 'engagement_promedio']]
y = df['volumen_posts']

# Dividir en entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Datos de entrenamiento: {len(X_train)} registros")
print(f"Datos de prueba: {len(X_test)} registros")

In [ ]:
# Entrenar modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Predicciones
y_pred_train = modelo.predict(X_train)
y_pred_test = modelo.predict(X_test)

# Evaluación del modelo
print("=" * 60)
print("EVALUACIÓN DEL MODELO")
print("=" * 60)
print(f"\nConjunto de Entrenamiento:")
print(f"  R² Score: {r2_score(y_train, y_pred_train):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"  MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")

print(f"\nConjunto de Prueba:")
print(f"  R² Score: {r2_score(y_test, y_pred_test):.4f}")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"  MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")

In [ ]:
# Importancia de características
importancias = pd.DataFrame({
    'Característica': ['Día Número', 'Día Semana', 'Es Evento', 'Hashtag', 'Engagement'],
    'Coeficiente': modelo.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)

print("\n" + "=" * 60)
print("IMPORTANCIA DE CARACTERÍSTICAS")
print("=" * 60)
print(importancias)

# Visualizar
plt.figure(figsize=(10, 6))
plt.barh(importancias['Característica'], importancias['Coeficiente'], color='teal')
plt.xlabel('Coeficiente', fontsize=12)
plt.title('Importancia de Características en el Modelo', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar predicciones vs reales
plt.figure(figsize=(12, 6))

plt.scatter(y_test, y_pred_test, alpha=0.6, color='blue', edgecolors='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Predicción Perfecta')

plt.xlabel('Volumen Real', fontsize=12)
plt.ylabel('Volumen Predicho', fontsize=12)
plt.title('Predicciones vs Valores Reales', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Predicción de Tendencias (Próxima Semana)

In [ ]:
# Generar predicciones para los próximos 7 días
fecha_inicio_prediccion = fecha_fin + timedelta(days=1)
fechas_futuras = pd.date_range(start=fecha_inicio_prediccion, periods=7, freq='D')

predicciones_futuras = []

for hashtag in hashtags:
    hashtag_code = pd.Categorical(pd.Series([hashtag]), categories=hashtags).codes[0]
    
    # Obtener engagement promedio histórico del hashtag
    engagement_historico = df[df['hashtag'] == hashtag]['engagement_promedio'].mean()
    
    for fecha in fechas_futuras:
        dia_numero = (fecha - df['fecha'].min()).days
        dia_semana = fecha.dayofweek
        
        # Asumir sin eventos (se puede modificar si se conocen eventos futuros)
        es_evento = 0
        
        # Crear características
        X_futuro = np.array([[dia_numero, dia_semana, es_evento, hashtag_code, engagement_historico]])
        
        # Predecir
        volumen_predicho = modelo.predict(X_futuro)[0]
        
        predicciones_futuras.append({
            'fecha': fecha,
            'hashtag': hashtag,
            'volumen_predicho': max(0, int(volumen_predicho))
        })

df_predicciones = pd.DataFrame(predicciones_futuras)

print("=" * 60)
print("PREDICCIONES PARA LOS PRÓXIMOS 7 DÍAS")
print("=" * 60)
print(df_predicciones)

In [ ]:
# Visualizar predicciones futuras
plt.figure(figsize=(15, 8))

for hashtag in hashtags:
    # Datos históricos (últimos 14 días)
    df_hist = df[df['hashtag'] == hashtag].tail(14)
    plt.plot(df_hist['fecha'], df_hist['volumen_posts'], 
             marker='o', label=f'{hashtag} (histórico)', linewidth=2)
    
    # Predicciones
    df_pred = df_predicciones[df_predicciones['hashtag'] == hashtag]
    plt.plot(df_pred['fecha'], df_pred['volumen_predicho'], 
             marker='s', linestyle='--', linewidth=2, alpha=0.7)

plt.axvline(x=fecha_fin, color='red', linestyle=':', linewidth=2, label='Hoy')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Volumen de Posts', fontsize=12)
plt.title('Predicción de Volumen de Posts para la Próxima Semana', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Análisis de Tendencias: ¿Qué Hashtags Están Subiendo o Bajando?

In [ ]:
# Calcular tendencia (pendiente) para cada hashtag
tendencias = []

for hashtag in hashtags:
    df_hashtag = df[df['hashtag'] == hashtag].sort_values('dia_numero')
    
    # Calcular tendencia usando últimos 30 días
    df_reciente = df_hashtag.tail(30)
    
    if len(df_reciente) > 1:
        X_trend = df_reciente['dia_numero'].values.reshape(-1, 1)
        y_trend = df_reciente['volumen_posts'].values
        
        modelo_tendencia = LinearRegression()
        modelo_tendencia.fit(X_trend, y_trend)
        
        pendiente = modelo_tendencia.coef_[0]
        
        # Calcular volumen promedio actual vs hace 30 días
        volumen_actual = df_reciente.tail(7)['volumen_posts'].mean()
        volumen_anterior = df_reciente.head(7)['volumen_posts'].mean()
        cambio_porcentual = ((volumen_actual - volumen_anterior) / volumen_anterior) * 100
        
        # Predicción promedio próxima semana
        prediccion_promedio = df_predicciones[df_predicciones['hashtag'] == hashtag]['volumen_predicho'].mean()
        
        tendencias.append({
            'hashtag': hashtag,
            'pendiente': pendiente,
            'cambio_porcentual': cambio_porcentual,
            'volumen_actual': volumen_actual,
            'prediccion_promedio': prediccion_promedio,
            'estado': 'SUBIENDO ↑' if pendiente > 0 else 'BAJANDO ↓'
        })

df_tendencias = pd.DataFrame(tendencias).sort_values('cambio_porcentual', ascending=False)

print("=" * 90)
print("ANÁLISIS DE TENDENCIAS")
print("=" * 90)
print(df_tendencias.to_string(index=False))

In [ ]:
# Visualización de tendencias
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Cambio porcentual
colors = ['green' if x > 0 else 'red' for x in df_tendencias['cambio_porcentual']]
axes[0].barh(df_tendencias['hashtag'], df_tendencias['cambio_porcentual'], color=colors, alpha=0.7)
axes[0].set_xlabel('Cambio Porcentual (%)', fontsize=12)
axes[0].set_title('Cambio en Volumen (Últimos 30 días)', fontsize=13, fontweight='bold')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Comparación volumen actual vs predicción
x = np.arange(len(df_tendencias))
width = 0.35

axes[1].barh(x - width/2, df_tendencias['volumen_actual'], width, 
             label='Volumen Actual (Promedio 7 días)', color='skyblue', alpha=0.8)
axes[1].barh(x + width/2, df_tendencias['prediccion_promedio'], width, 
             label='Predicción Promedio (Próxima semana)', color='orange', alpha=0.8)

axes[1].set_yticks(x)
axes[1].set_yticklabels(df_tendencias['hashtag'])
axes[1].set_xlabel('Volumen de Posts', fontsize=12)
axes[1].set_title('Volumen Actual vs Predicción', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Relación con Eventos del Mundo Real

In [ ]:
# Analizar impacto de eventos
print("=" * 90)
print("IMPACTO DE EVENTOS EN HASHTAGS")
print("=" * 90)

for fecha_evento, info_evento in eventos.items():
    print(f"\n📅 Evento: {info_evento['nombre']} ({fecha_evento.date()})")
    print(f"Hashtags afectados: {', '.join(info_evento['hashtags'])}")
    print("\nComparación de volumen:")
    
    for hashtag in info_evento['hashtags']:
        # Volumen en el evento
        vol_evento = df[(df['hashtag'] == hashtag) & (df['fecha'] == fecha_evento)]['volumen_posts'].values
        
        # Volumen promedio sin eventos
        vol_sin_evento = df[(df['hashtag'] == hashtag) & (df['es_evento'] == 0)]['volumen_posts'].mean()
        
        if len(vol_evento) > 0:
            incremento = ((vol_evento[0] - vol_sin_evento) / vol_sin_evento) * 100
            print(f"  {hashtag}:")
            print(f"    - Volumen en evento: {vol_evento[0]:,.0f}")
            print(f"    - Volumen promedio normal: {vol_sin_evento:,.0f}")
            print(f"    - Incremento: {incremento:.1f}%")

In [ ]:
# Visualizar efecto de eventos específicos
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, (fecha_evento, info_evento) in enumerate(eventos.items()):
    if idx < 4:
        ax = axes[idx]
        
        # Rango de fechas alrededor del evento (7 días antes y después)
        fecha_inicio_rango = fecha_evento - timedelta(days=7)
        fecha_fin_rango = fecha_evento + timedelta(days=7)
        
        for hashtag in info_evento['hashtags']:
            df_rango = df[(df['hashtag'] == hashtag) & 
                         (df['fecha'] >= fecha_inicio_rango) & 
                         (df['fecha'] <= fecha_fin_rango)]
            
            ax.plot(df_rango['fecha'], df_rango['volumen_posts'], 
                   marker='o', label=hashtag, linewidth=2)
        
        ax.axvline(x=fecha_evento, color='red', linestyle='--', linewidth=2, alpha=0.7)
        ax.set_title(f"{info_evento['nombre']}", fontsize=11, fontweight='bold')
        ax.set_xlabel('Fecha', fontsize=10)
        ax.set_ylabel('Volumen de Posts', fontsize=10)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Recomendaciones Estratégicas para Marcas

In [ ]:
# Generar recomendaciones basadas en análisis
print("=" * 90)
print("🎯 RECOMENDACIONES ESTRATÉGICAS")
print("=" * 90)

print("\n1. HASHTAGS DE ALTO POTENCIAL (Invertir contenido AHORA):")
print("-" * 90)
alto_potencial = df_tendencias[df_tendencias['cambio_porcentual'] > 0].head(3)
for idx, row in alto_potencial.iterrows():
    print(f"\n   {row['hashtag']}")
    print(f"   - Crecimiento: {row['cambio_porcentual']:.1f}%")
    print(f"   - Volumen esperado próxima semana: {row['prediccion_promedio']:,.0f} posts/día")
    print(f"   - Estado: {row['estado']}")
    print(f"   - Recomendación: INVERTIR - Crear contenido ahora para aprovechar la tendencia")

print("\n\n2. HASHTAGS EN DECLIVE (Evitar o replantear estrategia):")
print("-" * 90)
en_declive = df_tendencias[df_tendencias['cambio_porcentual'] < 0].tail(3)
for idx, row in en_declive.iterrows():
    print(f"\n   {row['hashtag']}")
    print(f"   - Decrecimiento: {row['cambio_porcentual']:.1f}%")
    print(f"   - Volumen esperado próxima semana: {row['prediccion_promedio']:,.0f} posts/día")
    print(f"   - Estado: {row['estado']}")
    print(f"   - Recomendación: PRECAUCIÓN - Diversificar o esperar rebote")

print("\n\n3. PRÓXIMOS EVENTOS A MONITOREAR:")
print("-" * 90)
print("\n   Para maximizar impacto, crear contenido 3-5 días ANTES de estos eventos:")
print("   - Conferencias tecnológicas → #TechTrends, #AIRevolution")
print("   - Eventos de salud/fitness → #HealthyLifestyle, #FitnessGoals")
print("   - Lanzamientos cripto/financieros → #CryptoNews")
print("   - Temporadas de compras → #DigitalMarketing, #Startup")

print("\n\n4. MEJOR MOMENTO PARA PUBLICAR:")
print("-" * 90)
mejores_dias = df.groupby('dia_semana')['volumen_posts'].mean().sort_values(ascending=False)
dias_nombre = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
print("\n   Días con mayor actividad de hashtags:")
for dia, volumen in mejores_dias.head(3).items():
    print(f"   - {dias_nombre[dia]}: {volumen:,.0f} posts promedio")

print("\n" + "=" * 90)

## 10. Dashboard Resumen

In [ ]:
# Crear dashboard visual completo
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Top hashtags por volumen predicho
ax1 = fig.add_subplot(gs[0, :])
top_predicciones = df_tendencias.sort_values('prediccion_promedio', ascending=False).head(5)
ax1.barh(top_predicciones['hashtag'], top_predicciones['prediccion_promedio'], 
         color='darkgreen', alpha=0.7)
ax1.set_xlabel('Volumen Predicho Promedio (Próxima Semana)', fontsize=11)
ax1.set_title('🚀 TOP 5 HASHTAGS CON MAYOR VOLUMEN PREDICHO', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Tendencias (subiendo vs bajando)
ax2 = fig.add_subplot(gs[1, 0])
subiendo = len(df_tendencias[df_tendencias['cambio_porcentual'] > 0])
bajando = len(df_tendencias[df_tendencias['cambio_porcentual'] < 0])
ax2.pie([subiendo, bajando], labels=['Subiendo ↑', 'Bajando ↓'], 
        autopct='%1.0f%%', colors=['lightgreen', 'lightcoral'], startangle=90)
ax2.set_title('Distribución de Tendencias', fontsize=11, fontweight='bold')

# 3. Engagement vs Volumen
ax3 = fig.add_subplot(gs[1, 1:])
for hashtag in hashtags[:5]:  # Top 5 para claridad
    df_hash = df[df['hashtag'] == hashtag]
    ax3.scatter(df_hash['volumen_posts'], df_hash['engagement_promedio'], 
               label=hashtag, alpha=0.6, s=50)
ax3.set_xlabel('Volumen de Posts', fontsize=10)
ax3.set_ylabel('Engagement Promedio', fontsize=10)
ax3.set_title('Relación Volumen-Engagement (Top 5)', fontsize=11, fontweight='bold')
ax3.legend(fontsize=8)
ax3.grid(True, alpha=0.3)

# 4. Mayor crecimiento
ax4 = fig.add_subplot(gs[2, :2])
top_crecimiento = df_tendencias.sort_values('cambio_porcentual', ascending=False).head(5)
colors_crec = ['green' if x > 0 else 'red' for x in top_crecimiento['cambio_porcentual']]
ax4.barh(top_crecimiento['hashtag'], top_crecimiento['cambio_porcentual'], 
         color=colors_crec, alpha=0.7)
ax4.set_xlabel('Cambio Porcentual (%)', fontsize=10)
ax4.set_title('📈 Mayor Cambio en Tendencia (30 días)', fontsize=11, fontweight='bold')
ax4.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax4.grid(True, alpha=0.3)

# 5. Métricas del modelo
ax5 = fig.add_subplot(gs[2, 2])
ax5.axis('off')
metricas_text = f"""
📊 MÉTRICAS DEL MODELO

R² Score: {r2_score(y_test, y_pred_test):.3f}

RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.0f}

MAE: {mean_absolute_error(y_test, y_pred_test):.0f}

Precisión: {'Alta' if r2_score(y_test, y_pred_test) > 0.7 else 'Media'}
"""
ax5.text(0.1, 0.5, metricas_text, fontsize=11, verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('DASHBOARD: PREDICCIÓN DE TENDENCIAS DE HASHTAGS', 
             fontsize=16, fontweight='bold', y=0.98)
plt.show()

## 11. Conclusiones

In [ ]:
print("=" * 90)
print("📋 CONCLUSIONES DEL ANÁLISIS")
print("=" * 90)

print("\n1. RENDIMIENTO DEL MODELO:")
print(f"   - El modelo alcanzó un R² de {r2_score(y_test, y_pred_test):.3f}, indicando {'buena' if r2_score(y_test, y_pred_test) > 0.7 else 'aceptable'} capacidad predictiva")
print(f"   - Error promedio (MAE): {mean_absolute_error(y_test, y_pred_test):.0f} posts")
print("   - Las características más influyentes son: engagement, eventos, y día")

print("\n2. INSIGHTS CLAVE:")
print(f"   - {subiendo} hashtags muestran tendencia CRECIENTE")
print(f"   - {bajando} hashtags muestran tendencia DECRECIENTE")
print("   - Los eventos aumentan el volumen de posts hasta 3x")
print("   - La relación volumen-engagement es fuertemente positiva")

print("\n3. VALOR PARA MARCAS:")
print("   ✓ Identificar hashtags emergentes ANTES que la competencia")
print("   ✓ Optimizar calendario de contenido según predicciones")
print("   ✓ Aprovechar eventos para maximizar alcance")
print("   ✓ Evitar inversión en hashtags en declive")

print("\n4. PRÓXIMOS PASOS:")
print("   → Integrar datos reales de APIs (Twitter, Instagram, TikTok)")
print("   → Mejorar modelo con features adicionales (sentimiento, ubicación)")
print("   → Crear sistema de alertas automáticas para hashtags emergentes")
print("   → Desarrollar dashboard interactivo en tiempo real")

print("\n" + "=" * 90)
print("\n✅ Análisis completado exitosamente!")
print("\nEste modelo permite a las marcas tomar decisiones data-driven sobre")
print("qué hashtags usar, cuándo publicar, y cómo aprovechar tendencias emergentes.")
print("\n" + "=" * 90)